# DREAMER Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from scipy.io import loadmat
import heartpy as hp

In [2]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score, ShuffleSplit, train_test_split

In [3]:
# def preprocessing_and_feature(data):
#     data_ECG = {}
#     for participant in range(0,23):
#         for video in range(0,18):
#             # load raw baseline and stimuli data for left and right
#             basl_l=raw['DREAMER'][0,0]['Data'][0,participant]['ECG'][0,0]['baseline'][0,0][video,0][:,0]
#             stim_l=raw['DREAMER'][0,0]['Data'][0,participant]['ECG'][0,0]['stimuli'][0,0][video,0][:,0]
#             basl_r=raw['DREAMER'][0,0]['Data'][0,participant]['ECG'][0,0]['baseline'][0,0][video,0][:,1]
#             stim_r=raw['DREAMER'][0,0]['Data'][0,participant]['ECG'][0,0]['stimuli'][0,0][video,0][:,1]
#             # process with neurokit
#             ecg_signals_b_l, info_b_l = nk.ecg_process(basl_l,sampling_rate=256)
#             ecg_signals_s_l, info_s_l = nk.ecg_process(stim_l,sampling_rate=256)
#             ecg_signals_b_r, info_b_r = nk.ecg_process(basl_r,sampling_rate=256)
#             ecg_signals_s_r, info_s_r = nk.ecg_process(stim_r,sampling_rate=256)
#             # divide stimuli features by baseline features
#             # would be interesting to compare classification accuracy when we
#             # don't do this
#             features_ecg_l=nk.ecg_intervalrelated(ecg_signals_s_l)/nk.ecg_intervalrelated(ecg_signals_b_l)
#             features_ecg_r=nk.ecg_intervalrelated(ecg_signals_s_r)/nk.ecg_intervalrelated(ecg_signals_b_r)
#             # average left and right features
#             # would be interesting to compare classification accuracy when we
#             # rather include both left and right features
#             features_ecg=(features_ecg_l+features_ecg_r)/2
#             if not len(data_ECG):
#                 data_ECG=features_ecg
#             else:
#                 data_ECG=pd.concat([data_ECG,features_ecg],ignore_index=True)
#     return data_ECG

In [4]:
raw = loadmat("data/DREAMER.mat")

In [5]:
def preprocessing_and_feature(data):
    data_ECG = {}
    ecgDf = pd.DataFrame()
    for participant in range(0,23):
        for video in range(0,18):
            # load raw baseline and stimuli data for left and right
            basl_l=raw['DREAMER'][0,0]['Data'][0,participant]['ECG'][0,0]['baseline'][0,0][video,0][:,0]
            stim_l=raw['DREAMER'][0,0]['Data'][0,participant]['ECG'][0,0]['stimuli'][0,0][video,0][:,0]
            basl_r=raw['DREAMER'][0,0]['Data'][0,participant]['ECG'][0,0]['baseline'][0,0][video,0][:,1]
            stim_r=raw['DREAMER'][0,0]['Data'][0,participant]['ECG'][0,0]['stimuli'][0,0][video,0][:,1]

            colName = "P" + str(participant+1) + "V" + str(video+1)
           
            ecgDf = pd.concat([ecgDf, pd.DataFrame({colName : stim_l})], axis=1)
            
    
    return ecgDf

In [6]:
# ecg = preprocessing_and_feature(raw)

In [7]:
# ecg.head()

In [8]:
# ecg.to_csv('data/DREAMER_ECG_Raw.csv')

In [9]:
def calculateBPMLib(seriesIn, colName):
    print(colName)
    fs = 256 #The example dataset was recorded at 256Hz
    
    series = seriesIn.copy()
#     print(series.shape)
    dropIdx = len(series.dropna())
    series = series.truncate(after=dropIdx-1)
    print(series.shape)

    minuteHz = 7680
    currIdx = 7680
    bpmList = []
    
#     if(len(series)<minuteHz):
#         print('ERROR0')
#         return []
    
    while currIdx < len(series):
#         print(currIdx)
        currSlice = series.iloc[currIdx-minuteHz:currIdx]
        currIdx += 256
        currSlice = np.array(currSlice.to_list())
        try:
            bpmList.append(hp.process(currSlice, fs)[1]['bpm'])
        except hp.exceptions.BadSignalWarning:
            bpmList.append(np.nan)

    return bpmList

In [10]:
# hrDf = pd.DataFrame()

# for col in ecg.columns:
#     hrDf = pd.concat([hrDf, pd.DataFrame({col : calculateBPMLib(ecg[col], col)})], axis=1)

In [11]:
# hrDf.shape

In [12]:

# count = 0

# for i in temp.iteritems():
#     if i[1]/364 > 0.7:
#         hrDf = hrDf.drop([i[0]], axis=1)
#         df_Participants_Data = df_Participants_Data.drop([i[0]], axis=1)
#         count+=1
# print(count)

In [13]:
df_Participants_Data.head()

NameError: name 'df_Participants_Data' is not defined

In [ ]:
df_Participants_Data.to_csv('data/DREAMER_emotLabels.csv', index=False)

In [ ]:
hrDf.to_csv('data/preprocessedDREAMER.csv', index=False)

In [14]:
lenList = []
for col in hrDf.columns:
    lenList.append(len(hrDf[col].dropna()))

NameError: name 'hrDf' is not defined

In [15]:
def Participants_Data(raw):
    cols = []
    # Create new dataframe with emotion, participant, and video data
    a=np.zeros((23,18,9),dtype=object)
    for participant in range(0,23):
        for video in range(0,18):
            cols.append("P" + str(participant+1) + "V" + str(video+1))
            a[participant,video,0]=raw['DREAMER'][0,0]['Data'][0,participant]['Age'][0][0][0]
            a[participant,video,1]=raw['DREAMER'][0,0]['Data'][0,participant]['Gender'][0][0][0]
            a[participant,video,2]=participant+1
            a[participant,video,3]=video+1
            a[participant,video,4]=['Searching for Bobby Fischer','D.O.A.', 'The Hangover', 'The Ring', '300',
                      'National Lampoon\'s VanWilder', 'Wall-E', 'Crash', 'My Girl', 'The Fly',
                      'Pride and Prejudice', 'Modern Times', 'Remember the Titans', 'Gentlemans Agreement',
                      'Psycho', 'The Bourne Identitiy', 'The Shawshank Redemption', 'The Departed'][video]
            a[participant,video,5]=['calmness', 'surprise', 'amusement', 'fear', 'excitement', 'disgust',
                      'happiness', 'anger', 'sadness', 'disgust', 'calmness', 'amusement',
                      'happiness', 'anger', 'fear', 'excitement', 'sadness', 'surprise'][video]
            a[participant,video,6]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreValence'][0,0][video,0].astype(float)
            a[participant,video,7]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreArousal'][0,0][video,0].astype(float)
            a[participant,video,8]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreDominance'][0,0][video,0].astype(float)
    b=pd.DataFrame(a.reshape((23*18,a.shape[2])),columns=['Age','Gender','Participant','Video','Video_Name','Target_Emotion','Valence','Arousal','Dominance'])
    
    c = pd.DataFrame(b[['Valence', 'Arousal', 'Dominance']].T.values, columns=cols, index=['Valence', 'Arousal', 'Dominance'])
    ## combine feature extraction dataframes with the new dataframe
    #all_data=pd.concat([data_EEG,data_ECG,b],axis=1)
    return c

In [16]:
df_Participants_Data = Participants_Data(raw)
df_Participants_Data.head()

,P1V1,P1V2,P1V3,P1V4,P1V5,P1V6,P1V7,P1V8,P1V9,P1V10,...,P23V9,P23V10,P23V11,P23V12,P23V13,P23V14,P23V15,P23V16,P23V17,P23V18
Valence,4.0,3.0,5.0,4.0,4.0,1.0,5.0,1.0,1.0,5.0,...,2.0,2.0,4.0,4.0,4.0,2.0,2.0,3.0,2.0,2.0
Arousal,3.0,3.0,4.0,3.0,4.0,2.0,4.0,2.0,2.0,3.0,...,2.0,5.0,1.0,2.0,3.0,2.0,2.0,3.0,2.0,4.0
Dominance,2.0,1.0,4.0,2.0,4.0,5.0,4.0,4.0,3.0,4.0,...,2.0,5.0,1.0,2.0,3.0,2.0,2.0,2.0,4.0,2.0


In [33]:
df_Participants_Data = pd.read_csv('data/ASCERTAIN_labels.csv')
df_Participants_Data.shape

FileNotFoundError: [Errno 2] No such file or directory: 'data/ASCERTAIN_labels.csv'

In [32]:
outList = []
for col in df_Participants_Data.columns:
    val = df_Participants_Data[col]['Valence']
    arousal = df_Participants_Data[col]['Arousal']
    if(val <=1.5 and arousal <= 1.5):
        outList.append(1)
    else:
        outList.append(0)

KeyError: 'Valence'

In [24]:
outDf = pd.DataFrame(outList, index=df_Participants_Data.columns)
outDf = outDf.T

In [26]:
# outDf.to_csv("finalData/DR_fullLabels.csv")

In [ ]:
df_Participants_Data.to_csv('data/DREAMER_labels_raw.csv')

In [ ]:
df_Participants_Data = Participants_Data(raw)
df_Participants_Data.head()

In [ ]:
df_Features.head()

In [ ]:
len(testList)

In [ ]:
df_Features = pd.DataFrame()
df_Features = preprocessing_and_feature(raw)

In [ ]:
cleanedDf = df_Features.dropna(axis=1, how='any')
cleanedDf.to_csv("data/preprocessedDREAMER.csv", index=False)

In [ ]:
labels = df_Participants_Data['Target_Emotion']
labels.shape

In [ ]:
cleanedDf = pd.read_csv("data/preprocessedDREAMER.csv")

In [ ]:
cvf = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

In [ ]:
rfModel = RandomForestClassifier(n_estimators=1000, random_state=0)
score = np.mean(cross_val_score(rfModel, cleanedDf, labels, cv=cvf))

In [ ]:
score

In [ ]:
cleanedDf.shape

# Labels

In [ ]:
df_Participants_Data.head()